# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 21

Group Members:
- Elbert Chailes (13522045)
- Juan Alfred Widjaya (13522073)
- Albert (13522081)
- William Glory Henderson (13522113)

## Import Libraries

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Import other libraries if needed

## Import Dataset

In [29]:
# Example of reading a csv file from a gdrive link

# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# # Don't forget to change the access to public
# from google.colab import files
df_additional_features_train = pd.read_csv ('https://drive.google.com/uc?id=1qYWAlxa_56A9xOXWVcnKlqMoSJMisjYg')
df_basic_features_train = pd.read_csv ('https://drive.google.com/uc?id=1TfswxBe7vTuFle0-q-6THJX3tcqkyf4c')
df_content_features_train = pd.read_csv('https://drive.google.com/uc?id=1kT4J_VxAHMJjkfY-WlQ_Bvrcza-zLLmO')
df_flow_features_train = pd.read_csv('https://drive.google.com/uc?id=1CH-2cpECHpfaUewvaQpqjNfhf3LxMZxQ')
df_labels_train = pd.read_csv('https://drive.google.com/uc?id=1snLxxb-irje8B4YqnsfjRx1S_Nv3Zh0W')
df_time_features_train = pd.read_csv('https://drive.google.com/uc?id=1gQADRwogQx20oolH9pn-CghflY8XwFKK')
df_UNSW_NB15_features_modified = pd.read_csv('https://drive.google.com/uc?id=19Vmv5ku_85Z9Uc3VjgF2OUdQTJrCwMR4')

df = pd.merge(df_additional_features_train, df_basic_features_train, on='id', how='inner')
df = pd.merge(df, df_content_features_train, on='id', how='inner')
df = pd.merge(df, df_flow_features_train, on='id', how='inner')
df = pd.merge(df, df_labels_train, on='id', how='inner')
df = pd.merge(df, df_time_features_train, on='id', how='inner')
df = df.drop('id', axis=1)
# df.to_csv('Tubes AI.csv', index=False)

# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [30]:
import seaborn as sns
import scipy.stats as stats

numerical = df.select_dtypes(include=['float64', 'int64']).columns
categorical = df.select_dtypes(include=['object', 'category']).columns

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [31]:
from sklearn.model_selection import train_test_split

df_copy = df.copy()

X = df_copy.drop(columns=['attack_cat', 'label'])
y = df_copy['attack_cat']

columns_to_keep = ['service', 'sttl']

X = X[columns_to_keep]
print(X.head())
# Split the data into training (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shape of the splits
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

  service   sttl
0       -   31.0
1     dns  254.0
2       -  254.0
3       -   31.0
4     dns    NaN
Training set size: 140272
Validation set size: 35069


# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer


def handle_missing_data(X, skewness_threshold=0.5):
    # Calculate skewness only for numeric columns
    numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
    skewness = X[numeric_cols].skew()

    middle_skewed = skewness[abs(skewness) <= skewness_threshold].index
    high_skewed = skewness[abs(skewness) > skewness_threshold].index

    imputers = {}

    # Create imputers based on skewness for each column
    for col in numeric_cols:
        strategy = 'mean' if col in middle_skewed else 'median'
        imputers[col] = SimpleImputer(strategy=strategy)

    # Fit the imputers for each column
    for col, imputer in imputers.items():
        imputer.fit(X[[col]])

    # Apply the imputers to fill missing values
    X_imputed = X.copy()
    for col, imputer in imputers.items():
        X_imputed[col] = imputer.transform(X[[col]])

    return X_imputed


### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [33]:
# import math
# num_columns = len(numerical_columns)
# num_rows = math.ceil(num_columns / 4)

# numerical_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# lower_threshold = X_train[numerical_columns].quantile(0.05)
# upper_threshold = X_train[numerical_columns].quantile(0.95)

# X_train_capped = X_train.copy()
# for col in numerical_columns:
#     X_train_capped[col] = np.clip(X_train_capped[col], lower_threshold[col], upper_threshold[col])

# plt.figure(figsize=(15, num_rows * 4))
# for i, col in enumerate(numerical_columns, 1):
#     plt.subplot(num_rows, 4, i)
#     sns.boxplot(X_train_capped[col])
#     plt.title(f'Box plot of {col}')
#     plt.tight_layout()

# plt.subplots_adjust(hspace=0.5)
# plt.show()

def outlier_remover(X, method='IQR', threshold=1.5):
    outlier_bounds = {}

    # Apply outlier removal only to numeric columns
    numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

    if method == 'IQR':
        # Calculate the outlier bounds using IQR
        for col in numeric_cols:
            Q1 = X[col].quantile(0.25)
            Q3 = X[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            outlier_bounds[col] = (lower_bound, upper_bound)

    X_outlier_removed = X.copy()

    # Clip outliers for each numeric column based on IQR bounds
    for col, (lower, upper) in outlier_bounds.items():
        X_outlier_removed[col] = X_outlier_removed[col].clip(lower, upper)

    return X_outlier_removed


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [34]:
def remove_duplicates(data):
  data.drop_duplicates(inplace=True)

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [35]:
# Write your code here




## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [36]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [37]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import RobustScaler

class CustomRobustScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = RobustScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        return self.scaler.transform(X)


### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder

class CustomOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, handle_unknown='ignore'):
        self.encoder = OneHotEncoder(handle_unknown=handle_unknown)

    def fit(self, X, y=None):
        """Fit the encoder to the data"""
        self.encoder.fit(X)
        return self

    def transform(self, X):
        """Apply the encoding to the data"""
        return self.encoder.transform(X).toarray()


### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [39]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class CustomNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, method='standardize'):
        self.method = method

    def fit(self, X, y=None):
        """
        Compute the statistics required for normalization (e.g., mean, std for standardization).
        """
        if self.method == 'standardize':
            self.mean_ = np.mean(X, axis=0)
            self.std_ = np.std(X, axis=0)
        elif self.method == 'minmax':
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        return self

    def transform(self, X):
        """
        Apply the normalization transformation to the data.
        """
        X_transformed = X.copy()

        if self.method == 'standardize':
            X_transformed = (X_transformed - self.mean_) / self.std_
        elif self.method == 'minmax':
            X_transformed = (X_transformed - self.min_) / (self.max_ - self.min_)

        return X_transformed

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [40]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Separate numerical and categorical columns
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

numeric_pipeline = Pipeline([
    ('missing_data', FunctionTransformer(lambda X: handle_missing_data(X, skewness_threshold=0.5))),
    ('missing_outlier', FunctionTransformer(lambda X: outlier_remover(X, method='IQR', threshold=1.5))),
    ('scaler', CustomRobustScaler()),
    # ('normalizer', CustomNormalizer(method='standardize'))
])

# Categorical preprocessing pipeline
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Step 3: Combine both pipelines into a ColumnTransformer
preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_features),
    ('categorical', categorical_pipeline, categorical_features)
],)

# from sklearn.model_selection import train_test_split
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer

# # Separate numerical and categorical columns
# numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
# categorical_features = X.select_dtypes(include=['object', 'category']).columns

# numeric_pipeline = Pipeline([
#     ('missing_data', FunctionTransformer(lambda X: handle_missing_data(X, skewness_threshold=0.5))),
#     ('missing_outlier', FunctionTransformer(lambda X: outlier_remover(X, method='IQR', threshold=1.5))),
#     ('scaler', CustomRobustScaler()),
#     # ('normalizer', CustomNormalizer(method='standardize'))
# ])

# # Categorical preprocessing pipeline
# categorical_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Step 3: Combine both pipelines into a ColumnTransformer
# preprocessor = ColumnTransformer([
#     ('numeric', numeric_pipeline, numeric_features),
#     ('categorical', categorical_pipeline, categorical_features)
# ],)

In [42]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [43]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [44]:
# from sklearn.neighbors import KNeighborsClassifier
# from src.KNN import OptimizedKNN
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score, classification_report, log_loss
# from sklearn.preprocessing import LabelEncoder
# model_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('classifier', KNeighborsClassifier(n_neighbors=15))
# ])

# model_pipeline.fit(X_train, y_train)
# y_val_pred = model_pipeline.predict(X_val)
# print(classification_report(y_val, y_val_pred))
# f1_macro = f1_score(y_val, y_val_pred, average='macro')
# print(f'F1 Score (Macro Average) on Validation Set: {f1_macro}')

In [45]:
# from sklearn.neighbors import KNeighborsClassifier
# from src.KNN import OptimizedKNN
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score, classification_report, log_loss
# from sklearn.preprocessing import LabelEncoder
# from src.ModelLoader import ModelLoader

# model_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
# ])

# model_pipeline.fit(X_train, y_train)
# X_train_transformed = model_pipeline.transform(X_train)
# X_val_transformed = model_pipeline.transform(X_val)

# label_encoder = LabelEncoder()
# # y_combined = pd.concat([y_train, y_val])

# # Fit the label encoder on combined data
# # label_encoder.fit(y_combined)
# # y_train_encoded = label_encoder.transform(y_train)
# # y_val_encoded = label_encoder.transform(y_val)
# # Fit the pipeline

# knn = OptimizedKNN(k=11, distance_metric='euclidean')
# knn.fit(X_train_transformed, y_train)
# # print(y_combined)
# ModelLoader.save(knn, "test_knn.pkl")
# y_val_pred = knn.predict(X_val_transformed)
# # attack_cat_labels = label_encoder.inverse_transform(y_val_pred)
# print(classification_report(y_val, y_val_pred))
# f1_macro = f1_score(y_val, y_val_pred, average='macro')
# print(f'F1 Score (Macro Average) on Validation Set: {f1_macro}')



## B. Naive Bayes

In [46]:
# # Type your code hfrom sklearn.neighbors import KNeighborsClassifier
# from src.NaiveBayes import NaiveBayes
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score, classification_report, log_loss
# model_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
# ])

# # Fit the pipelin
# model_pipeline.fit(X_train, y_train)

# nb = NaiveBayes()
# nb.fit(X_train, y_train)
# y_val_pred = nb.predict(X_val)
# print(classification_report(y_val, y_val_pred))
# f1_macro = f1_score(y_val, y_val_pred, average='macro')
# print(f'F1 Score (Macro Average) on Validation Set: {f1_macro}')

In [47]:
# from sklearn.naive_bayes import GaussianNB

# from sklearn.neighbors import KNeighborsClassifier
# from src.KNN import OptimizedKNN
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score, classification_report, log_loss
# from sklearn.preprocessing import LabelEncoder
# model_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('classifier', GaussianNB())
# ])

# model_pipeline.fit(X_train, y_train)
# y_val_pred = model_pipeline.predict(X_val)
# print(classification_report(y_val, y_val_pred))
# f1_macro = f1_score(y_val, y_val_pred, average='macro')
# print(f'F1 Score (Macro Average) on Validation Set: {f1_macro}')

## C. ID3

In [48]:
from src.ID3 import ID3
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.preprocessing import LabelEncoder
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
])

model_pipeline.fit(X_train, y_train)

X_train_transformed_from_pipeline = model_pipeline.named_steps['preprocessing'].fit_transform(X_train)
X_train_densed_from_pipeline = X_train_transformed_from_pipeline.toarray()
X_train_densed_from_pipeline_int = X_train_densed_from_pipeline.astype(float)

print(len(X_train_densed_from_pipeline))
print(type[X_train_densed_from_pipeline])
for x in X_train_densed_from_pipeline[7886]:
  print(type(x))
id3_model = ID3()
id3_model.fit(X_train_densed_from_pipeline_int, y_train)

X_val_transformed_from_pipeline = model_pipeline.transform(X_val)
X_val_densed_from_pipeline = X_val_transformed_from_pipeline.toarray()

y_val_pred = id3_model.predict(X_val_densed_from_pipeline)

print(classification_report(y_val, y_val_pred))
f1_macro = f1_score(y_val, y_val_pred, average='macro')
print(f'F1 Score (Macro Average) on Validation Set: {f1_macro}')

140272
type[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])]
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

                precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00       400
      Backdoor       1.00      0.01      0.02       349
           DoS       0.40      0.00      0.00      2453
      Exploits       0.69      0.51      0.59      6679
       Fuzzers       0.24      0.93      0.38      3637
       Generic       0.98      0.93      0.95      8000
        Normal       0.99      0.73      0.84     11200
Reconnaissance       0.83      0.00      0.01      2098
     Shellcode       0.00      0.00      0.00       227
         Worms       0.00      0.00      0.00        26

      accuracy                           0.64     35069
     macro avg       0.51      0.31      0.28     35069
  weighted avg       0.78      0.64      0.64     35069

F1 Score (Macro Average) on Validation Set: 0.2786005645462815


## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [49]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [50]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`

In [51]:
columns_to_keep = ['ct_state_ttl','ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm','ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'sttl', 'swin','dwin']

df_test_copy = df_test[columns_to_keep]
X_test_transformed = model_pipeline.transform(df_test_copy)

knn_loaded = ModelLoader.load("test_knn.pkl")

# Step 4: Predict the attack categories for the test data
y_test_pred = knn_loaded.predict(X_test_transformed)

# Step 5: Convert the predictions from label encoding to the original category names (if needed)
attack_cat_labels = y_test_pred

# Step 6: Create the submission file in the required format
# Assuming the 'id' column is preserved in df_test for creating the submission
submission_df = pd.DataFrame({
    'id': range(len(df_test_copy)), 
    'attack_cat': attack_cat_labels
})

# Step 7: Save the submission file as CSV
submission_df.to_csv('submission.csv', index=False)

KeyError: "None of [Index(['service'], dtype='object')] are in the [columns]"